In [ ]:
!git clone https://github.com/thewh1teagle/optispeech -b he-v1
%cd optispeech

In [ ]:
%env RYE_INSTALL_OPTION=--yes
!curl -sSf https://rye.astral.sh/get | bash

!bash -c "$(wget -O - https://apt.llvm.org/llvm.sh)"
!sudo apt install cmake nvtop htop p7zip-full aria2 -y
!sudo ln -s /usr/bin/clang-18 /usr/bin/clang
!sudo ln -s /usr/bin/clang++-18 /usr/bin/clang++

In [ ]:
!/root/.rye/shims/rye sync
!/root/.rye/shims/rye add torchaudio==2.3.1

In [ ]:
!wget https://github.com/thewh1teagle/optispeech/releases/download/v0.1.0/espeak-ng-data.7z
!7z x espeak-ng-data.7z
import os
os.environ['ESPEAK_DATA_PATH'] = os.path.join(os.getcwd(), 'espeak-ng-data')

In [ ]:
!aria2c -x16 -s16 https://github.com/thewh1teagle/optispeech/releases/download/v0.1.0/last.ckpt.7z
!aria2c -x16 -s16 https://github.com/thewh1teagle/optispeech/releases/download/v0.1.0/saspeech-v3.7z
!7z x saspeech-v3.7z
!7z x last.ckpt.7z

In [37]:
!sed -i "s|/root/home/optispeech/|$(pwd)/|g" ./data/saspeech/val.txt ./data/saspeech/train.txt

In [ ]:
!.venv/bin/python -m optispeech.train \
    experiment="saspeech-he" \
    model.train_args.evaluate_utmos=false \
    data.batch_size=8 \
    data.num_workers=2 \
    data.train_filelist_path="data/saspeech/train.txt" \
    data.valid_filelist_path="data/saspeech/val.txt" \
    paths.log_dir="data/saspeech/logs" \
    callbacks.model_checkpoint.every_n_epochs=1  \
    callbacks.model_checkpoint.save_on_train_epoch_end=True \
    ckpt_path="last.ckpt"